In [1]:
# recent gameweek
gameweek = 3

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
from scipy import stats

pd.set_option('max_columns',100)

In [3]:
# check whether team total cost is less than allowed
def is_within_budget(team, cost_threshold):
    return team['now_cost'].sum()/10.0 <= cost_threshold        

In [4]:
# create a custom discrete probability distribution
def discrete_probabilities(nr_elements):
    x = np.arange(nr_elements)
    weights = np.arange(nr_elements,0,-1) / nr_elements
    probabilities = weights / np.sum(weights)
    return stats.rv_discrete(values=(x, probabilities))

In [5]:
# replace a player from a team that is over the budget
def downgrade_team(team,nr_goalkeepers,nr_field):
    # choose a random number between 0-10 with custom weighting
#    custom_probability_generator_team = discrete_probabilities(11)
    custom_ix = custom_probability_generator_team.rvs()
    # find ix of player to be replaced
    replace_ix = team.sort_values(by='valuePoints metric').index[custom_ix]
    # find playing position of player to be replaced
    element_type = team.loc[replace_ix,'element_type']
    # order potential replacements based on valuePoints
    players_ordered = df.loc[(~df.index.isin(team.index)) & (df['element_type']==element_type) & minGames]\
                                                .sort_values(by='valuePoints metric', ascending=False).head(nr_field)
    if element_type > 1:
        custom_probability_generator_new_player = discrete_probabilities(nr_field)
    else:
        custom_probability_generator_new_player = discrete_probabilities(nr_goalkeepers)
    custom_ix = custom_probability_generator_new_player.rvs()
    new_ix = players_ordered.index[custom_ix]
    team = team.drop(replace_ix)
    team = team.append(df.loc[new_ix])
    team = team.sort_values(by='element_type')
    # total cost of dream team
    total_cost = team['now_cost'].sum()/10.0
    # total points for dream team (best player's points doubled for captaincy)
    total_points = team['adjusted points per game'].sum() + team['adjusted points per game'].max()
    return team, total_cost, total_points

In [6]:
def upgrade_team(team, cash_available):
    input_team = team # NOT USED?!
    changes = 0
    for i in range(11):
        player_ix = team.sort_values(by='valuePoints metric').index[i]
        player_cost = df.loc[player_ix,'now_cost'] / 10.0
        element_type = df.loc[player_ix,'element_type']
        better_player_ix = df.loc[(~df.index.isin(team.index)) & (df['element_type']==element_type) & minGames \
                                 & (df['now_cost']/10.0 <= (player_cost+cash_available))]\
                                    .sort_values(by='valuePoints metric', ascending=False).index[0]

        if df.loc[better_player_ix,'valuePoints metric'] > df.loc[player_ix,'valuePoints metric']:
            team = team.drop(player_ix)
            team = df.loc[better_player_ix:better_player_ix].append(team)
            changes = 1

        # total cost of team
        total_cost = team['now_cost'].sum()/10.0
        cash_available = cost_threshold - total_cost
        # total points for dream team (best player's points doubled for captaincy)
        total_points = team['adjusted points per game'].sum() + team['adjusted points per game'].max()  
        team = team.sort_values(by='element_type')
        return team, cash_available, total_cost, total_points, changes

In [8]:
# fetch FPL data
filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
#filepath = 'data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)
df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,team_name,games played,adjusted points,xG,xG_points,xG_week3,goals_week3,xPoints week 3,clean_sheet_points,xGA_week3,cleansheet_week3,xA,xA_points,xA_week3,assists_week3,adjusted points per game,xPoints week 1,xPoints week 2,next1_xP,next5_xP,next10_xP,value,valuePoints metric
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,NaN,NaN,0,37605,4.0,NaN,-1,1,-2,2,0.0,549,225,NaN,NaN,0,3,0.5,-0.5,0,Mesut,0.0,0,0,0.0,549,225,False,0.0,548,225,0,NaN,NaN,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.8,False,NaN,a,1,3,0.0,548,224,0,2081,411,33087,3138,0.0,0.0,Özil,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,75.0,75.0,0,39476,NaN,NaN,0,0,-1,1,0.0,444,174,NaN,NaN,0,2,0.4,-0.4,0,Sokratis,0.0,0,0,0.0,458,175,False,0.0,453,175,0,Calf injury - 75% chance of playing,2020-09-11T11:00:08.600094Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,NaN,d,1,3,0.0,415,161,0,9750,26,15817,437,0.0,0.0,Sokratis,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,14,100.0,100.0,0,41270,NaN,NaN,0,0,0,0,3.2,233,67,4.0,NaN,0,2,1.2,0.2,1,David,0.7,3,0,2.3,270,93,False,20.2,209,84,91,NaN,2020-09-10T14:30:11.632386Z,55,0,0,NaN,0,NaN,41270.jpg,1.0,0,0,Luiz Moreira Marinho,1.0,False,NaN,a,1,3,0.0,509,192,2,12279,3134,34411,2522,0.1,0.4,David Luiz,0,Arsenal,2.000000,3.461468,0.0,0.0,0.0,0.0,1.199148,1.461468,3.0,0.0,0.0,0.0,0.0,0.0,1.730734,NaN,1.000000,2.368348,1.954587,2.033910,0.314679,0.737987
4,1,0,44,NaN,NaN,1,54694,NaN,NaN,0,0,-1,1,52.6,39,27,6.0,NaN,0,3,5.2,4.2,2,Pierre-Emerick,4.7,4,1,19.4,35,19,False,68.8,59,25,270,NaN,NaN,119,0,0,1.0,0,NaN,54694.jpg,4.7,0,0,Aubameyang,29.3,False,NaN,a,1,3,73.0,29,14,14,230384,32737,1352199,110427,0.4,1.2,Aubameyang,1,Arsenal,2.978723,10.940715,0.5,2.5,0.0,0.0,2.049787,0.544164,3.0,0.0,0.5,1.5,0.0,0.0,3.672954,3.818731,4.149569,3.867314,3.738578,4.111039,0.308652,1.064736
5,0,0,0,75.0,75.0,0,58822,NaN,NaN,-1,1,-2,2,0.0,438,171,NaN,NaN,0,2,0.4,-0.4,0,Cédric,0.0,0,0,0.0,453,172,False,0.0,448,172,0,Calf injury - 75% chance of playing,2020-09-23T09:00:14.881983Z,48,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.3,False,NaN,d,1,3,0.0,408,158,0,1382,32,23705,2283,0.0,0.0,Cédric,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 1

Choose team formation and find an initial team with the best possible players.

In [18]:
# set up formation (number of defenders, midfielders and forwards, 1 goalkeeper assumed)
formation = [4,4,2]
cost_threshold = 83
# choose the best goalkeeper
team_goalkeeper = df[goalkeepers & minGames].sort_values(by='adjusted points per game', ascending=False).head(1)
# choose formation[0] best defenders
team_defenders = df[defenders & minGames].sort_values(by='adjusted points per game', ascending=False).head(formation[0])
# choose formation[1] best midfielders
team_midfielders = df[midfielders & minGames].sort_values(by='adjusted points per game', ascending=False).head(formation[1])
# choose formation[2] best forwards
team_forwards = df[forwards & minGames].sort_values(by='adjusted points per game', ascending=False).head(formation[2])
# create initial team
team = team_goalkeeper.append(team_defenders).append(team_midfielders).append(team_forwards)
# total cost of dream team
total_cost = team['now_cost'].sum()/10.0
# total points for dream team (best player's points doubled for captaincy)
total_points = team['adjusted points per game'].sum() + team['adjusted points per game'].max()
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print()
print(team['web_name'])
print()
print('Total cost: ' + str(total_cost))
print()
print('Total points per gameweek: ' + str(total_points))
print()
print('Is this team within budget?')
print(is_within_budget(team, cost_threshold))

#save this team as an initial starting point for searches
dream_team = team

Best team with formation 4-4-2:

id
12        Martínez
155          Digne
442        Masuaku
41           Mings
255      Robertson
251           Mané
272      De Bruyne
254          Salah
33       Hourihane
388           Kane
166    Richarlison
Name: web_name, dtype: object

Total cost: 87.2

Total points per gameweek: 94.24481546397442

Is this team within budget?
False


Start an iterative process looking for the best team with given formation and budget.

In [19]:
# createa a custom discrete probability distribution for numbers 0-10
custom_probability_generator_team = discrete_probabilities(11)

In [20]:
iterations = 100
nr_goalkeepers = 20
nr_field = 30

best_points = 0
for i in range(iterations):
    team = dream_team
    while ~is_within_budget(team, cost_threshold):
        team, total_cost, total_points = downgrade_team(team,nr_goalkeepers,nr_field)
    
    cash_available = cost_threshold - total_cost
    changes = 1
    while changes==1:
        team, cash_available, total_cost, total_points, changes = upgrade_team(team, cash_available)
    
    if total_points > best_points:
        best_team = team
        best_points = total_points
        best_cost = total_cost
        
    if i%10 == 0:
        print(str(i) + '/' + str(iterations))
    
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

0/100
10/100
20/100
30/100
40/100
50/100
60/100
70/100
80/100
90/100
id
12        Martínez
46           Konsa
155          Digne
442        Masuaku
41           Mings
251           Mané
105       Jorginho
272      De Bruyne
33       Hourihane
388           Kane
166    Richarlison
Name: web_name, dtype: object

Total cost: 77.7

Total points per gameweek: 91.74630824504624


In [13]:
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

Best team with formation 3-5-2:
id
12      Martínez
46         Konsa
442      Masuaku
41         Mings
33     Hourihane
251         Mané
272    De Bruyne
254        Salah
105     Jorginho
68        Maupay
388         Kane
Name: web_name, dtype: object

Total cost: 82.2

Total points per gameweek: 92.42102944685925


In [17]:
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

Best team with formation 3-4-3:
id
12          Martínez
442          Masuaku
155            Digne
41             Mings
272        De Bruyne
105         Jorginho
251             Mané
33         Hourihane
388             Kane
166      Richarlison
164    Calvert-Lewin
Name: web_name, dtype: object

Total cost: 80.6

Total points per gameweek: 94.28931496922252


In [21]:
print('Best team with formation ' + str(formation[0]) + '-' + str(formation[1])+ '-' + str(formation[2]) + ':')
print(best_team['web_name'])
print()
print('Total cost: ' + str(best_cost))
print()
print('Total points per gameweek: ' + str(best_points))

Best team with formation 4-4-2:
id
12        Martínez
46           Konsa
155          Digne
442        Masuaku
41           Mings
251           Mané
105       Jorginho
272      De Bruyne
33       Hourihane
388           Kane
166    Richarlison
Name: web_name, dtype: object

Total cost: 77.7

Total points per gameweek: 91.74630824504624
